# Yolo V4 on SageMaker Notebook

### Download pre-trained model and darknet code

1. Open t2.medium in SageMaker notebook
2. In the terminal of notebook

In [ ]:
bash
cd SageMaker
git clone https://github.com/AlexeyAB/darknet
cd darknet
wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights
wget https://pjreddie.com/media/files/yolov3.weights

## Switch SageMaker notebook to p3.2xlarge

In [ ]:
bash
source activate tensorflow_p36
sudo bash -c 'echo "/usr/local/lib64" > /etc/ld.so.conf.d/opencv.conf'
sudo ldconfig -v

In [ ]:
cd SageMaker/darknet/
mkdir build-release
cmake ..
make
make install

In [ ]:
cd ..
# test yolo v3
./darknet detect cfg/yolov3.cfg yolov3.weights data/dog.jpg
./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights -ext_output data/dog.jpg

# SageMaker Notebook Error on P3 Debug

### case1: missing the library on p3.2xlarge 

libopencv_dnn.so.4.2: cannot open shared object file: No such file or directory

In [ ]:
(tensorflow_p36) [ec2-user@ip-172-16-30-255 darknet]$ ./darknet detect cfg/yolov3.cfg yolov3.weights data/dog.jpg
./darknet: error while loading shared libraries: libopencv_dnn.so.4.2: cannot open shared object file: No such file or directory

### Solutions for this error

1. Find the folder containing the shared library `sudo find / -name "libopencv_dnn.so.*"`
2. Create a file called `/etc/ld.so.conf.d/opencv.conf` and write to it the path to the folder where the binary is stored
3. run `sudo ldconfig -v`

In [ ]:
sudo find / -name "libopencv_dnn.so.*"

In [ ]:
(tensorflow_p36) [ec2-user@ip-172-16-90-69 darknet]$ sudo find / -name "libopencv_dnn.so.*"
/usr/local/lib64/libopencv_dnn.so.4.2.0
/usr/local/lib64/libopencv_dnn.so.4.2
/home/ec2-user/anaconda3/pkgs/opencv-3.3.0-py27_blas_openblas_204/lib/libopencv_dnn.so.3.3
/home/ec2-user/anaconda3/pkgs/opencv-3.3.0-py27_blas_openblas_204/lib/libopencv_dnn.so.3.3.0

In [ ]:
sudo bash -c 'echo "/usr/local/lib64" > /etc/ld.so.conf.d/opencv.conf'
sudo ldconfig -v

# Appendix

In [ ]:
## uname -a
(tensorflow_p36) [ec2-user@ip-172-16-30-255 darknet]$ uname -a
Linux ip-172-16-30-255 4.14.171-105.231.amzn1.x86_64 #1 SMP Thu Feb 27 23:49:15 UTC 2020 x86_64 x86_64 x86_64 GNU/Linux

## cmake output
(tensorflow_p36) [ec2-user@ip-172-16-30-255 build-release]$ cmake ..
-- The C compiler identification is GNU 4.8.5
-- The CXX compiler identification is GNU 4.8.5
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Setting build type to 'Release' as none was specified.
-- Looking for a CUDA compiler
-- Looking for a CUDA compiler - /usr/local/cuda/bin/nvcc
-- The CUDA compiler identification is NVIDIA 10.0.130
-- Check for working CUDA compiler: /usr/local/cuda/bin/nvcc
-- Check for working CUDA compiler: /usr/local/cuda/bin/nvcc -- works
-- Detecting CUDA compiler ABI info
-- Detecting CUDA compiler ABI info - done
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Looking for pthread_create
-- Looking for pthread_create - not found
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: TRUE
-- Found CUDA: /usr/local/cuda-10.0 (found version "10.0")
-- Autodetected CUDA architecture(s):  7.0
-- Building with CUDA flags: -gencode;arch=compute_70,code=sm_70
-- Your setup supports half precision (it requires CC >= 7.0)
-- Found OpenCV: /usr/local (found version "4.2.0")
-- Found Stb: /home/ec2-user/SageMaker/darknet/3rdparty/stb/include
-- Found OpenMP_C: -fopenmp (found version "3.1")
-- Found OpenMP_CXX: -fopenmp (found version "3.1")
-- Found OpenMP: TRUE (found version "3.1")
--   ->  darknet is fine for now, but uselib_track has been disabled!
--   ->  Please rebuild OpenCV from sources with CUDA support to enable it
-- Found CUDNN: /usr/local/cuda-10.0/include (found version "7.5.1")
-- CMAKE_CUDA_FLAGS: -gencode arch=compute_70,code=sm_70 --compiler-options " -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -Wno-deprecated-declarations -Wno-write-strings -DGPU -DCUDNN -DOPENCV -fPIC -fopenmp -Ofast "
-- ZED SDK not found
-- Configuring done
-- Generating done
-- Build files have been written to: /home/ec2-user/SageMaker/darknet/build-release